## 유방암 환자 분류: 딥러닝을 이용한 이진 분류

이 노트북은 Scikit-learn에 내장된 위스콘신 유방암 데이터셋을 사용하여 악성(malignant)과 양성(benign) 종양을 분류하는 딥러닝 모델을 구축합니다. 이진 분류 문제에 딥러닝을 적용하는 전체 과정을 다룹니다.

### 1. 라이브러리 임포트

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pickle

### 2. 데이터 로드 및 탐색
Scikit-learn의 `load_breast_cancer` 함수를 사용하여 데이터를 로드하고, Pandas DataFrame으로 변환하여 내용을 쉽게 확인합니다. 그 후, 데이터를 훈련 세트와 테스트 세트로 분할합니다.

In [ ]:
def load_data():
    """데이터를 로드하고 훈련/테스트 세트로 분할합니다."""
    cancer = load_breast_cancer()
    X = pd.DataFrame(cancer.data, columns=cancer.feature_names)
    y = cancer.target # 0: 악성(malignant), 1: 양성(benign)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        test_size=0.2, 
        random_state=123, 
        stratify=y # 원본 데이터의 클래스 비율을 유지
    )
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = load_data()

print("--- 데이터 확인 ---")
print("훈련 데이터 형태:", X_train.shape)
print("테스트 데이터 형태:", X_test.shape)
print("\n첫 5개 훈련 데이터 샘플:")
print(X_train.head())

### 3. 데이터 전처리: 특성 스케일링
각 특성(feature)의 값 범위가 다르기 때문에, 모델이 안정적으로 학습할 수 있도록 `StandardScaler`를 사용하여 모든 특성을 표준화합니다.

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("--- 스케일링 후 데이터 확인 ---")
print("평균:", X_train_scaled.mean(axis=0).round(2))
print("표준편차:", X_train_scaled.std(axis=0).round(2))

### 4. 딥러닝 모델 구축
이진 분류를 위한 `Sequential` 모델을 정의합니다.

- **은닉층**: `relu` 활성화 함수를 사용하는 여러 개의 `Dense` 층으로 구성됩니다.
- **출력층**: 두 개의 클래스(악성/양성) 중 하나일 확률을 출력해야 하므로, 하나의 뉴런과 `sigmoid` 활성화 함수를 사용합니다. Sigmoid 함수는 출력을 0과 1 사이의 값으로 변환하여 확률처럼 해석할 수 있게 합니다.

모델 컴파일 시:
- **Optimizer**: `rmsprop`
- **Loss Function**: 이진 분류 문제이므로 `binary_crossentropy`를 사용합니다.
- **Metrics**: `accuracy`를 사용하여 모델 성능을 측정합니다.

In [ ]:
def create_model():
    model = models.Sequential([
        layers.Input(shape=(X_train_scaled.shape[1],)), # 입력 특성의 수
        layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(1, activation='sigmoid'),
    ])

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

model = create_model()
model.summary()

### 5. 모델 훈련
모델을 훈련시키면서 `ModelCheckpoint` 콜백을 사용하여 검증 손실(`val_loss`)이 가장 낮은 최적의 모델을 파일로 저장합니다. 이렇게 하면 훈련이 끝난 후 가장 성능이 좋았던 시점의 모델을 사용할 수 있습니다.

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='암환자분류_최적모델.keras', # 모델 저장 경로
        save_best_only=True,          # 가장 좋은 모델만 저장
        monitor='val_loss'            # 검증 손실을 기준으로 판단
    )
]

history = model.fit(X_train_scaled, y_train, 
                    epochs=20, 
                    validation_data=(X_test_scaled, y_test),
                    callbacks=callbacks, 
                    batch_size=100,
                    verbose=1)

# 훈련 과정 기록을 pickle로 저장
with open("암환자분류_훈련기록.pkl", "wb") as file:
    pickle.dump(history.history, file)

### 6. 훈련 과정 시각화
훈련 및 검증 과정에서의 손실과 정확도 변화를 그래프로 그려, 모델이 어떻게 학습되었는지, 과대적합은 발생하지 않았는지 확인합니다.

In [ ]:
def plot_history(history):
    plt.figure(figsize=(12, 5))

    # 손실 그래프
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # 정확도 그래프
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()

plot_history(history)

### 7. 모델 평가
훈련이 완료된 후, 저장된 최적의 모델을 불러와 최종 성능을 평가합니다.

In [ ]:
# 저장된 최적 모델 로드
best_model = keras.models.load_model('암환자분류_최적모델.keras')

print("--- 최종 모델 평가 (최적 모델) ---")
train_loss, train_acc = best_model.evaluate(X_train_scaled, y_train)
test_loss, test_acc = best_model.evaluate(X_test_scaled, y_test)

print(f"훈련셋 손실값: {train_loss:.4f}, 정확도: {train_acc:.4f}")
print(f"테스트셋 손실값: {test_loss:.4f}, 정확도: {test_acc:.4f}")